```
Copyright 2024 The HIVEX Authors. All Rights Reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or  implied.
See the License for the specific language governing permissions and
limitations under the License.
```

# Example Training using ML-Agents and Torch MADDPG

This notebook is a modified version of the Tennis.ipynb from the Udacity nano-degree [Deep Reinforcement Learning](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893)
<br>
Note:
Note:
1. Install the dependencies as described in the README.md.
2. Select the correct kernel for this jupyter notebook at the top right.

## Import Libraries

In [111]:
import numpy as np
import random
import copy
from collections import namedtuple, deque
import time

# torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# ML-Agents
from mlagents_envs.base_env import ActionTuple

# hivex
from hivex.training.framework_wrappers.wrapper_utils import initialize_unity_environment

In [112]:
def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)

class Actor(nn.Module):
    """Actor (Policy) Model."""

    # 400, 300
    def __init__(self, state_size, action_size, seed, fc1_units=256, fc2_units=256):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(Actor, self).__init__()
        self.state_size = state_size
        self.seed = torch.manual_seed(seed)

        self.bn1 = nn.BatchNorm1d(state_size)
        self.fc1 = nn.Linear(state_size, fc1_units)

        self.bn2 = nn.BatchNorm1d(fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)

        self.bn3 = nn.BatchNorm1d(fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)

        self.reset_parameters()

    def reset_parameters(self):
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)

    """
    def forward(self, state):
        #Build an actor (policy) network that maps states -> actions.
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        return F.tanh(self.fc3(x))
    """

    def forward(self, state):
        #Build an actor (policy) network that maps states -> actions.
        if len(state) == self.state_size:
            state = state.unsqueeze(0)

        x = F.relu(self.fc1(self.bn1(state)))
        x = F.relu(self.fc2(self.bn2(x)))
        return F.tanh(self.fc3(self.bn3(x)))


class Critic(nn.Module):
    """Critic (Value) Model."""

    def __init__(self, state_size, action_size, seed, fcs1_units=256, fc2_units=256):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fcs1_units (int): Number of nodes in the first hidden layer
            fc2_units (int): Number of nodes in the second hidden layer
        """
        super(Critic, self).__init__()
        self.seed = torch.manual_seed(seed)

        self.fcs1 = nn.Linear(state_size, fcs1_units)
        self.bns = nn.BatchNorm1d(fcs1_units)

        self.fc2 = nn.Linear(fcs1_units + action_size, fc2_units)
        self.fc3 = nn.Linear(fc2_units, 1)

        self.reset_parameters()

    def reset_parameters(self):
        self.fcs1.weight.data.uniform_(*hidden_init(self.fcs1))
        self.fc2.weight.data.uniform_(*hidden_init(self.fc2))
        self.fc3.weight.data.uniform_(-3e-3, 3e-3)
    """
    def forward(self, state, action):
        #Build a critic (value) network that maps (state, action) pairs -> Q-values.
        xs = F.relu(self.fcs1(state))
        x = torch.cat((xs, action), dim=1)
        x = F.relu(self.fc2(x))
        return self.fc3(x)
    """

    def forward(self, state, action):
        #Build a critic (value) network that maps (state, action) pairs -> Q-values.
        xs = F.relu(self.fcs1(state))
        xs = self.bns(xs)
        x = torch.cat((xs, action), dim=1)
        x = F.relu(self.fc2(x))
        return self.fc3(x)

In [113]:
BUFFER_SIZE = int(1e5)  # replay buffer size # original value: 1e5
BATCH_SIZE = 512        # minibatch size
GAMMA = 0.999           # discount factor
TAU = 1e-2              # for soft update of target parameters
LR_ACTOR = 1e-3         # learning rate of the actor # original value: 1e-4
LR_CRITIC = 1e-3        # learning rate of the critic # original value: 1e-3
WEIGHT_DECAY = 0        # L2 weight decay

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Agent():
    """Interacts with and learns from the environment."""

    def __init__(self, state_size, action_size, random_seed):
        """Initialize an Agent object.

        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
            random_seed (int): random seed
        """

        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(random_seed)

        # Actor Network (w/ Target Network)
        self.actor_local = Actor(state_size, action_size, random_seed).to(device)
        self.actor_target = Actor(state_size, action_size, random_seed).to(device)
        self.actor_optimizer = optim.Adam(self.actor_local.parameters(), lr=LR_ACTOR)

        # Critic Network (w/ Target Network)
        self.critic_local = Critic(state_size, action_size, random_seed).to(device)
        self.critic_target = Critic(state_size, action_size, random_seed).to(device)
        self.critic_optimizer = optim.Adam(self.critic_local.parameters(), lr=LR_CRITIC, weight_decay=WEIGHT_DECAY)

        # Noise process
        self.noise = OUNoise(action_size, random_seed)

        # Replay memory
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, random_seed)

    def step(self, state, action, reward, next_state, done):
        """Save experience in replay memory, and use random sample from buffer to learn."""
        # Save experience / reward
        self.memory.add(state, action, reward, next_state, done)

        # Learn, if enough samples are available in memory
        if len(self.memory) > BATCH_SIZE:
            experiences = self.memory.sample()
            self.learn(experiences, GAMMA)


    def act(self, state, add_noise=True):
        """Returns actions for given state as per current policy."""
        state = torch.from_numpy(state).float().to(device)
        self.actor_local.eval()
        with torch.no_grad():
            action = self.actor_local(state).cpu().data.numpy()
        self.actor_local.train()
        if add_noise:
            action += self.noise.sample()
        return np.clip(action, -1, 1)

    def reset(self):
        self.noise.reset()

    def learn(self, experiences, gamma):
        """Update policy and value parameters using given batch of experience tuples.
        Q_targets = r + γ * critic_target(next_state, actor_target(next_state))
        where:
            actor_target(state) -> action
            critic_target(state, action) -> Q-value

        Params
        ======
            experiences (Tuple[torch.Tensor]): tuple of (s, a, r, s', done) tuples
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences

        # ---------------------------- update critic ---------------------------- #
        # Get predicted next-state actions and Q values from target models
        actions_next = self.actor_target(next_states)
        Q_targets_next = self.critic_target(next_states, actions_next)
        # Compute Q targets for current states (y_i)
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))
        # Compute critic loss
        Q_expected = self.critic_local(states, actions)
        critic_loss = F.mse_loss(Q_expected, Q_targets)
        # Minimize the loss
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        # ---------------------------- update actor ---------------------------- #
        # Compute actor loss
        actions_pred = self.actor_local(states)
        actor_loss = -self.critic_local(states, actions_pred).mean()
        # Minimize the loss
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        # ----------------------- update target networks ----------------------- #
        self.soft_update(self.critic_local, self.critic_target, TAU)
        self.soft_update(self.actor_local, self.actor_target, TAU)
        self.actor_loss = actor_loss.data
        self.critic_loss = critic_loss.data

    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target

        Params
        ======
            local_model: PyTorch model (weights will be copied from)
            target_model: PyTorch model (weights will be copied to)
            tau (float): interpolation parameter
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

class OUNoise:
    """Ornstein-Uhlenbeck process."""

    def __init__(self, size, seed, mu=0., theta=0.15, sigma=0.05):
        """Initialize parameters and noise process."""
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.seed = random.seed(seed)
        self.reset()

    def reset(self):
        """Reset the internal state (= noise) to mean (mu)."""
        self.state = copy.copy(self.mu)

    def sample(self):
        """Update internal state and return it as a noise sample."""
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.array([random.random() for i in range(len(x))])
        self.state = x + dx
        return self.state

class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.
        Params
        ======
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  # internal memory (deque)
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)

    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)

    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)

        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

In [114]:
class maddpg:
    def __init__(self, state_size, action_size, random_seed, num_agents):
        self.action_size = action_size
        self.num_agents = num_agents
        self.agents = [Agent(state_size, action_size, random_seed) for i in range(num_agents)]
        self.shared_memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, random_seed)

    def step(self, states, actions, rewards, next_states, dones):
        for i in range(self.num_agents):
            self.agents[i].step(states[0][i,:], actions[i,:], rewards[i], next_states[0][i,:], dones[i])
        self.shared_memory.add(states, actions, rewards, next_states, dones)

        """
        if len(self.shared_memory) > BATCH_SIZE:
            for i in range(self.num_agents):
                experiences = self.shared_memory.sample()
                self.agents[i].learn(experiences, GAMMA)
        """


    def act(self, states, add_noise=True):
        actions = np.zeros([self.num_agents, self.action_size])
        for i in range(self.num_agents):
            actions[i, :] = self.agents[i].act(np.squeeze(states)[i], add_noise)
        return actions

    def save_weights(self):
        for i in range(self.num_agents):
            torch.save(self.agents[i].actor_local.state_dict(), 'agent{}_checkpoint_actor.pth'.format(i+1))
            torch.save(self.agents[i].critic_local.state_dict(), 'agent{}_checkpoint_critic.pth'.format(i+1))


    def reset(self):
        for agent in self.agents:
            agent.reset()

In [115]:
def ddpg(agent, env, num_agents, n_episodes=1000, max_steps=1000):
    """
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        print_every (int): print scores every n-th episode
    """
    """
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))
    """
    scores = []
    average_scores_list = []
    scores_window = deque(maxlen=100)
    training_starting_time = time.time()
    
    brain_name = list(env.behavior_specs)[0]

    #memory = ReplayBuffer(action_size=4, BUFFER_SIZE=int(1e5), BATCH_SIZE=128, random_seed=0)
    
    episode = 0

    # loop through all episodes
    for i_episode in range(1, n_episodes+1):
        episode += 1
        # get time current time
        episode_starting_time = time.time()
        
        env_info = env.reset()                                 # reset the environment      
        decision_steps, terminal_steps = env.get_steps(brain_name)  
        states = decision_steps.obs                            # get the current state (for each agent)        
        agent.reset()         
        score = np.zeros(num_agents)                           # initialize the score (for each agent)
        

        step = 0
        # loop through all time steps in episode
        while True:
            step += 1
            # define actions
            actions = agent.act(states)

            if len(actions) > 0:
                if isinstance(actions[0], tuple):
                    action_tuple = ActionTuple(
                        continuous=np.array([c[0] for c in actions]),
                        discrete=np.array([c[1] for c in actions]),
                    )
                else:
                    if actions[0].dtype == np.float32:
                        action_tuple = ActionTuple(continuous=np.array(actions))
                    else:
                        action_tuple = ActionTuple(discrete=np.array(actions))
                env.set_actions(brain_name, action_tuple)
            
            # get next states, rewards and done from environment
            env.step()                                          # send all actions to tne environment
            decision_steps, terminal_steps = env.get_steps(brain_name)
            next_states = decision_steps.obs                   # get next state (for each agent)
            rewards = decision_steps.reward                    # get reward (for each agent)


            dones = [False for agent_id in range(num_agents)]
            if len(terminal_steps.interrupted) > 0:
                for agent_id in terminal_steps.agent_id:
                    dones[agent_id] = True
            
            # agents take step
            agent.step(states, actions, rewards, next_states, dones)
            
            #print(rewards)
            
            score += rewards                                   # update the score (for each agent)
            states = next_states                               # roll over states to next time step

            if np.any(dones):                                  # exit loop if episode finished 
                break
            
            print('\rEpisode {}\tStep: {}\tScore: {}'.format(i_episode, step, score), end="")

            if step >= max_steps:
                break
        
        score_max = np.max(score)
        scores.append(score_max)
        scores_window.append(score_max)
        average_score = np.mean(scores_window)
        average_scores_list.append(average_score)

        print('\rEpisode {}\tAverage Score: {:.3f}'.format(i_episode, np.mean(scores_window)), end="")  

        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage score: {:.3f}'.format(i_episode , average_score))

        if average_score >= 0.5:
            agent.save_weights()
            print("\rSolved in episode: {} \tAverage score: {:.3f}".format(i_episode , average_score))
            break
        
        """
        elapsed_training_time = time.time() - training_starting_time
        elapsed_training_time_str = time.strftime('%Mm%Ss', time.gmtime(elapsed_training_time))
        
        elapsed_episode_time = time.time() - episode_starting_time
        elapsed_episode_time_str = time.strftime('%Mm%Ss', time.gmtime(elapsed_episode_time))
        
        print('\rTime elapsed since training started: {}'.format(elapsed_training_time_str))
        print('\rEpisode {} took: {}'.format(i_episode, elapsed_episode_time_str))
 
        """
    
    return scores

## Initialize Environment

In [116]:
unity_env = initialize_unity_environment(
    worker_id=0, hivex_env_tag="WindFarmControl", no_graphics=True
)

## Run Training

In [117]:
agent = maddpg(state_size=6, action_size=1, random_seed=0, num_agents=8)
scores = ddpg(agent, unity_env, num_agents=8, n_episodes=1, max_steps=100)

Episode 1	Step: 1	Score: [0.         0.07821101 0.         0.         0.         0.
Episode 1	Step: 2	Score: [0.         0.15642202 0.         0.         0.         0.
Episode 1	Step: 3	Score: [0.         0.23463303 0.         0.         0.         0.
Episode 1	Step: 4	Score: [0.         0.31284404 0.         0.         0.         0.
Episode 1	Step: 5	Score: [0.         0.39105505 0.         0.         0.         0.
Episode 1	Step: 6	Score: [0.         0.46926606 0.         0.         0.         0.
Episode 1	Step: 7	Score: [0.         0.54747707 0.         0.         0.         0.
Episode 1	Step: 8	Score: [0.         0.62568808 0.         0.         0.         0.
Episode 1	Step: 9	Score: [0.         0.70389909 0.         0.         0.         0.
Episode 1	Step: 10	Score: [0.         0.7821101  0.         0.         0.         0.
Episode 1	Step: 11	Score: [0.         0.8603211  0.         0.         0.         0.
Episode 1	Step: 12	Score: [0.         0.93853211 0.         0.         0. 